In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
def init_weights(shape):
    init = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init)

In [7]:
def init_bias(shape):
    init = tf.constant(0.1,shape=shape)
    return tf.Variable(init)

In [10]:
def conv2d(x,W):
    # x ----> [batch , height , width , channels]
    # w ----> [filter height , filter width , channel in , channel out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1] , padding = 'SAME')

In [9]:
def max_pool_2b2(x):
    # x ----> [batch , height , width , channels]
    return tf.nn.max_pool(x , ksize = [1,2,2,1] , strides= [1,2,2,1],padding='SAME')

In [19]:
def conv_layer(input_x , shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

In [45]:
def full_conn_layer(inp_layer , size):
    inp_size = int(inp_layer.get_shape()[1])
    W = init_weights([inp_size,size])
    B = init_bias([size])
    return tf.matmul(inp_layer,W) + B   

In [62]:
X = tf.placeholder(tf.float32 , shape=[None,784])
Y_true = tf.placeholder(tf.float32 , shape=[None,10])
x_img = tf.reshape(X , [-1,28,28,1])

conv1 = conv_layer(x_img , shape = [5,5,1,32])
conv1_pool = max_pool_2b2(conv1)

conv2 = conv_layer(conv1_pool , shape = [5,5,32,64])
conv2_pool = max_pool_2b2(conv2)

conv3 = conv_layer(conv2_pool , shape = [5,5,64,32])
conv3_pool = max_pool_2b2(conv3)

conv_flat = tf.reshape(conv3_pool,[-1,4*4*32])
fcn = tf.nn.relu(full_conn_layer(conv_flat , 1024))

In [63]:
fcn.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'Relu_52:0' shape=(?, 1024) dtype=float32>>

In [64]:
hold_prob = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(fcn , keep_prob=hold_prob)

In [65]:
y_pred = full_conn_layer(dropout,10)

In [66]:
#LOSS
cross_entr = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_true , logits=y_pred))

In [67]:
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train = opt.minimize(cross_entr)
init = tf.global_variables_initializer()

In [74]:
import tqdm 
epochs = 500

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in (range(epochs)):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        sess.run(train , feed_dict = {X:batch_x , Y_true : batch_y , hold_prob:0.5})
        
        if i%10:
            
            match = tf.equal(tf.argmax(y_pred,1) , tf.argmax(Y_true , 1))
            
            acc = tf.reduce_mean(tf.cast(match,tf.float32))
            
            print(str(i) + "   " + str(sess.run(acc,feed_dict = {X:mnist.test.images , Y_true : mnist.test.labels , hold_prob : 1.0})))

1   0.1405
2   0.1726
3   0.1718
4   0.252
5   0.2945
6   0.2502
7   0.234
8   0.2447
9   0.2924
11   0.2804
12   0.3132
13   0.3883
14   0.4661
15   0.4974
16   0.487
17   0.4649
18   0.4633
19   0.4812
21   0.5563
22   0.5746
23   0.6096
24   0.6505
25   0.6757
26   0.6936
27   0.7132
28   0.7279
29   0.731
31   0.736
32   0.7486
33   0.7581
34   0.7739
35   0.7788
36   0.7781
37   0.7818
38   0.7887
39   0.8034
41   0.8194
42   0.8187
43   0.815
44   0.8148
45   0.8043
46   0.8017
47   0.8132
48   0.836
49   0.8492
51   0.8568
52   0.8467
53   0.8436
54   0.8536
55   0.8641
56   0.8669
57   0.8702
58   0.8728
59   0.8794
61   0.8768
62   0.88
63   0.8759
64   0.8631
65   0.8717
66   0.8784
67   0.8847
68   0.8865
69   0.881
71   0.8854
72   0.8887
73   0.8887
74   0.8929
75   0.8984
76   0.906
77   0.9084
78   0.91
79   0.9139
81   0.9136
82   0.9064
83   0.901
84   0.9015
85   0.9056
86   0.9073
87   0.9074
88   0.9056
89   0.9068
91   0.9165
92   0.9204
93   0.925
94   0.9249
95  